In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha5"

In [ ]:
open Deedle
open Plotly.NET

In [ ]:
let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

In [ ]:
data?Value
|> Series.observations
|> fun xy -> Chart.Line(xy, UseWebGL = true)

Going through the news the only thing indicating why there should have been such a dip in the energy network could have been interruption in the greater network due to the Krim crisis. This doesn't really check out, though.

In [ ]:
data?Value
|> Series.sampleTime (TimeSpan.FromDays(1.)) Direction.Forward
|> Series.mapValues Stats.median
|> Series.observations
|> fun xy -> Chart.Line(xy, UseWebGL = true)

In [ ]:
let data_2018 =
    data
    |> Frame.filterRows (fun idx rs -> idx.Year > 2017 && idx.Year < 2019)

data_2018?Value
|> Series.observations
|> fun xy -> Chart.Line(xy, UseWebGL = true)

In [ ]:
let predicateWeekOfYear (weeks: int list) (dt: DateTime) =
    let cal = System.Globalization.CultureInfo.InvariantCulture.Calendar
    let weekOfYear = cal.GetWeekOfYear(dt, Globalization.CalendarWeekRule.FirstDay, DayOfWeek.Monday)
    List.contains weekOfYear weeks

data_2018?Value
|> Series.observations
|> Seq.filter (fst >> (predicateWeekOfYear [10; 11; 12; 13]))
|> fun xy -> Chart.Line(xy, UseWebGL = true)